# imports

In [1]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
import seaborn as sns
import random
from scipy.stats import pearsonr
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from ShaniBA.GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import common_processing_feature_phenotype_matrices, editSampleNames,process_sample_matrix,compute_distance_matrix_general
from SampleFileFunctions import *

from tunneltoamazondb import getengine
from pandas import concat, read_csv, Series, read_sql


MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'31052018'

# final function to generate BD_FD sample conversion file:


Function updated=16.3.18 Change to matching BD-FD samples based on UserID only method! enables a little bit more samples with postHGF info
this function was copied to SampleFileFunctions.py - USE AND UPDATE THERE!

In [ ]:
def gen_BD_FD_conversion_file(mergeOn,readCountFolder):
    
    import time
    cdate=str(time.strftime("%d%m%Y"))
    cdate
    
    ## extract blood DNA information:
    BDsamples = read_sql('select  DnaID,UserID,storageDT from Lab.hostdna', getengine())
    BDsamples=BDsamples.rename(columns={'UserID':'oldUserID'})
    BDsamples['DnaID_num']=BDsamples['DnaID'].str.replace('BD','').astype(np.float)
    emptyBDs=BDsamples[BDsamples['DnaID'].isnull()]

    print 'extracted list of  %s BD samples from DB' %len(BDsamples)
    print 'last BD index is %s' %BDsamples['DnaID_num'].max()

    BDsamples=BDsamples[BDsamples['DnaID'].notnull()] #remove rows with no BD
    print 'number of rows without BD=%s-those rows were removed' %len(emptyBDs)

    countingBDs=pd.DataFrame(BDsamples['DnaID_num'].value_counts()) #counting repeating BDs
    repeatingBDs=countingBDs[countingBDs['DnaID_num']>1]
    BDsamples=BDsamples.drop_duplicates(subset='DnaID') #removing BD duplicates
    print 'BD samples repeating more than once=%s, duplicates were removed' %list(repeatingBDs.index)

    print 'final number of rows should be the initial list minus empty BD rows and minus duplicate BD rows\nand equall to the last BD index'
    print 'final number of rows is %s' %len(BDsamples)
    
    ## correct UserIDs using correct RegNums:
    #open the file with corrected userIDs for each BD number based on Daphna's correction for registration number:
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples'
    CleanCorrectBloodDNASamples=pd.read_pickle(file1)
    BD_correctReg=CleanCorrectBloodDNASamples[['DnaID','original registration code','correct registration code','correction status (reg number)']]

    
    # merge BD list with correct RegNum list:
    BDwithCorrectReg=pd.merge(BDsamples,BD_correctReg,how='left',left_on='DnaID',right_on='DnaID')
    BDwithCorrectReg['correct registration code']=BDwithCorrectReg['correct registration code'].fillna('unknown')
    BDwithCorrectReg['correction status (reg number)']=BDwithCorrectReg['correction status (reg number)'].fillna('unknown')
    print 'original BD sample list length is %s' %len(BDsamples)
    print 'sample list length after merging is %s (SHOULD BE THE SAME AS ABOVE!)' %len(BDwithCorrectReg)

    # extract UserID-regNum from the database
    UserIDs_RegNum=read_sql('select UserID,RegistrationCode from pnp.users', getengine())

    #merge new UserID to BD sample according to the corrected regNum
    BDwithCorrectUserID=pd.merge(BDwithCorrectReg,UserIDs_RegNum,how='left', 
                                 left_on='correct registration code',
                                right_on='RegistrationCode')
    #give old UserIDs and regNum to samples that don't have correct regNum:
    noCorrectReg=len(BDwithCorrectUserID[BDwithCorrectUserID['correct registration code']=='unknown'])
    print '%s samples dont have correct registration code and are given old UserID' %noCorrectReg
    BDwithCorrectUserID['UserID']=np.where(BDwithCorrectUserID['correct registration code']=='unknown',
                                      BDwithCorrectUserID['oldUserID'],BDwithCorrectUserID['UserID'])


    print 'sample number is still %s' %len(BDwithCorrectUserID)

    BDwithCorrectUserID=BDwithCorrectUserID[['DnaID','storageDT', 'UserID','correct registration code','correction status (reg number)']]

    #add regNum according to userID and use it only for samples with unknown regNum:
    BDwithCorrectUserIDandRegs=pd.merge(BDwithCorrectUserID,UserIDs_RegNum,how='left',
                                       left_on='UserID',right_on='UserID')
    BDwithCorrectUserIDandRegs['correct registration code']=np.where(BDwithCorrectUserIDandRegs['correct registration code']=='unknown',
                                      BDwithCorrectUserIDandRegs['RegistrationCode'],BDwithCorrectUserIDandRegs['correct registration code'])

    BDwithCorrectUserIDandRegs=BDwithCorrectUserIDandRegs.drop('RegistrationCode',axis=1)
    
    ## generate BD_year and date:
    BDwithCorrectUserIDandRegs['storageDT']=BDwithCorrectUserIDandRegs['storageDT'].astype(str)
    BDwithCorrectUserIDandRegs['Blood_Year']=BDwithCorrectUserIDandRegs['storageDT'].str.split('-').str[0]
    BDwithCorrectUserIDandRegs['Blood_Date']=BDwithCorrectUserIDandRegs['storageDT'].str.split(' ').str[0]
    BDwithCorrectUserIDandRegs['UserID']=BDwithCorrectUserIDandRegs['UserID'].astype(str)
    BDwithCorrectUserIDandRegs['UserID']=BDwithCorrectUserIDandRegs['UserID'].str.split('.').str[0]
    BDwithCorrectUserIDandRegs['UserID_bYear']=BDwithCorrectUserIDandRegs['UserID'].astype(str).str.cat(BDwithCorrectUserIDandRegs['Blood_Year'],sep='_')
    BDwithCorrectUserIDandRegs['UserID_bDate']=BDwithCorrectUserIDandRegs['UserID'].astype(str).str.cat(BDwithCorrectUserIDandRegs['Blood_Date'],sep='_')

    ## extract FD table
    FDsamples = read_sql('select  DnaID,UserID,connectionID,storageDT,SPID,isGenotek,Blacklisted from Lab.dna', getengine())
    print 'extracted list of  %s FD samples from DB' %len(FDsamples)
    
    ## generate FD year and date:
    FDsamples['storageDT']=FDsamples['storageDT'].astype(str)
    FDsamples['FD_Year']=FDsamples['storageDT'].str.split('-').str[0]
    FDsamples['FD_Date']=FDsamples['storageDT'].str.split(' ').str[0]
    FDsamples['UserID']=FDsamples['UserID'].astype(str)
    FDsamples['UserID']=FDsamples['UserID'].str.split('.').str[0]
    FDsamples['UserID_fYear']=FDsamples['UserID'].astype(str).str.cat(FDsamples['FD_Year'],sep='_')
    FDsamples['UserID_fDate']=FDsamples['UserID'].astype(str).str.cat(FDsamples['FD_Date'],sep='_')
    
    ### add read number information to FD samples:
    #read count info can be taken from either AllSeqProjects (new) or Metabolon2 (old) folder:
    if readCountFolder=='Metabolon2':
        f1='/net/mraid08/export/jafar/Microbiome/Analyses/Metabolon2/DFOut/ReadCountDF.dat'
    else:
        f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountSpidDF.dat'
    ReadCountDF=pd.read_pickle(f1)

    #check if there are FD samples repeating twice:
    maxRepeatsPerFD=ReadCountDF.reset_index()['FD'].value_counts().sort_values().max()
    print 'read count information for %s samples was loaded' %len(ReadCountDF)
    print 'maximal repeats per FD samples is %s - SHOULD BE ONE! IF NOT, NEED TO CHECK IT' %maxRepeatsPerFD
    #extract only important information to merge with FD table:
    ReadCountDFsmall=ReadCountDF.reset_index()[['FD','PostHGF','PostSubSamp']]
    #merge read counts info with FD table:
    FDsamples_withReads=pd.merge(FDsamples,ReadCountDFsmall,how='left',left_on='DnaID',right_on='FD')
    FDsamples_withReads=FDsamples_withReads.drop('FD',axis=1)
    #### try to merge unmerged samples based on SPID - CURRENTLY NOT IMPLEMENTED AS IT SEEMS TO GIVE WRONG INFO###
#     addInfo=0
#     for n in FDsamples_withReads[FDsamples_withReads['PostHGF'].isnull()].index:
#         SPID=FDsamples_withReads.loc[n,'SPID']
#         if SPID in list(ReadCountDF['SPID']):
#             postHGF=ReadCountDF[ReadCountDF['SPID']==SPID].loc[:,'PostHGF']
#             PostSubSamp=ReadCountDF[ReadCountDF['SPID']==SPID].loc[:,'PostSubSamp']
#             FDsamples_withReads.loc[n,'PostHGF']=postHGF.iloc[0]
#             FDsamples_withReads.loc[n,'PostSubSamp']=PostSubSamp.iloc[0]
#             addInfo=addInfo+1
#     print 'postHGF info was added based on SPID and not FD to %s samples' %addInfo
    
    
    
    ## save BD and FD files
    print 'saving BD and FD lists to excel files...'
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/BDfile_%s.xlsx' %cdate
    BDwithCorrectUserID.to_excel(file1)

    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/FDfile_RCfolder%s_%s.xlsx' %(readCountFolder,cdate) 
    FDsamples_withReads.to_excel(file2)
    
    print 'merging BD and FD samples...'
    
    BD_FD_converter=pd.merge(BDwithCorrectUserIDandRegs,FDsamples_withReads,how='left',left_on=['UserID','UserID_b%s' %mergeOn],
                        right_on=['UserID','UserID_f%s' %mergeOn])
    
    BD_FD_converter=BD_FD_converter.rename(columns={'DnaID_x':'BD','DnaID_y':'FD'})
    
    print 'after merging based on UserID and year, there are %s matches and %s BD samples without \
    a match' %(len(BD_FD_converter[BD_FD_converter['FD'].notnull()]),
          len(BD_FD_converter[BD_FD_converter['FD'].isnull()]))
    
    ### merging unmatched BDs to FDs based on UserID only:
    ### new strategy started on 16.3.18
    
    print 'merging unmatched samples based on UserID only...'
    BDwithNoFDs=BD_FD_converter[BD_FD_converter['FD'].isnull()]
    BDwithFDs=BD_FD_converter[BD_FD_converter['FD'].notnull()]


    converterLength=len(BD_FD_converter)
    noPostHGF=len(BD_FD_converter[BD_FD_converter['PostHGF'].notnull()])
    uniqueBDwithPostHGF=BD_FD_converter[BD_FD_converter['PostHGF'].notnull()]['BD'].nunique()
    print 'coverter length is %s' %converterLength
    print 'number of rows with noPostHGF info is %s' %noPostHGF
    print 'number of unique BDs with postHGF info is %s' %uniqueBDwithPostHGF
    
    BD_FD_converter['Comment']=''
    for BD in list(BDwithNoFDs['BD']):
        BDdf=BDwithNoFDs[BDwithNoFDs['BD']==BD]
        UserID=list(BDdf.loc[:,'UserID'])[0]
        n=BDdf.index
        BDtoMerge=BDwithCorrectUserIDandRegs[BDwithCorrectUserIDandRegs['DnaID']==BD]
    #     print n.astype(int),BD, UserID
    #     print 'length of BDtoMerge is %s and should be 1' %len(BDtoMerge)
        correspondingFDdf=FDsamples_withReads[FDsamples_withReads['UserID']==UserID]
        if len(correspondingFDdf)>0:
            newMerge=pd.merge(BDtoMerge,correspondingFDdf,how='left',
                              left_on='UserID',right_on='UserID')
            newMerge['Comment']='Not the same year!!'
            newMerge=newMerge.rename(columns={'DnaID_x':'BD','DnaID_y':'FD'})
    #         print newMerge
    #         print 'number of corresponding FD is %s' %len(correspondingFDdf)
    #         print 'len of newMerge is %s' %len(newMerge)
    #         print 'len of converter before changes is %s' %len(BD_FD_converter)
            BD_FD_converter=BD_FD_converter.drop(n,axis=0)
            BD_FD_converter=pd.concat([BD_FD_converter,newMerge]) 
    #         print 'len of converter after changes is %s' %len(BD_FD_converter)
    #         print BD_FD_converter[BD_FD_converter['BD']==BD]
    
    converterLength=len(BD_FD_converter)
    noPostHGF=len(BD_FD_converter[BD_FD_converter['PostHGF'].isnull()])
    uniqueBDwithPostHGF=BD_FD_converter[BD_FD_converter['PostHGF'].notnull()]['BD'].nunique()
    print 'converter length is %s' %converterLength
    print 'number of rows with noPostHGF info after new merging is %s' %noPostHGF
    print 'number of unique BDs with postHGF info after new merging is %s' %uniqueBDwithPostHGF
    
    ### remove rows with no FD number:
    print 'removing rows with no FD sample:'
    final_BD_FD_converter=BD_FD_converter[BD_FD_converter['FD'].notnull()]
    print 'n samples after removal=%s' %len(final_BD_FD_converter)
    
    
    
    ### save relevant columns only
    final_BD_FD_converter=final_BD_FD_converter[['BD', 'Blood_Year', 'Blood_Date','Comment', 'FD', 'FD_Year','FD_Date', u'SPID', u'UserID',
                                            'correct registration code','correction status (reg number)','UserID_bYear', 'UserID_fYear',
                                             'UserID_bDate', 'UserID_fDate',u'connectionID', u'isGenotek',
                                                 'Blacklisted','PostHGF','PostSubSamp']]    
    print 'I checked multyBDperFD pairs in file final_BD_FD_converter_mergedOnYear_07032018 (see section 4 below) and saw that trying to resolve those pairs by using dates instead of years was not proved helpful. so i will leave these pairs as is.'
    ## add StudyType
    #get studyTypeID and UserID information:
    studyTypeSPIDs = read_sql('select  * from pnp.spid', getengine())
    
    studyTypeSPIDs['SPID_Year']=studyTypeSPIDs['Start_date'].astype(str).str.split('-').str[0]
    studyTypeSPIDs['UserID']=studyTypeSPIDs['UserID'].astype(str)
    studyTypeSPIDs['UserID']=studyTypeSPIDs['UserID'].str.split('.').str[0]
    studyTypeSPIDs['UserID_SPIDYear']=studyTypeSPIDs['UserID'].astype(str).str.cat(studyTypeSPIDs['SPID_Year'],sep='_')

    studyTypeSPIDs_small=studyTypeSPIDs[['StudyTypeID','UserID_SPIDYear']]
    
    studyTypelists=studyTypeSPIDs_small.groupby('UserID_SPIDYear').agg(lambda x: x.unique().tolist())
    
    final_BD_FD_converter=pd.merge(final_BD_FD_converter,studyTypelists,how='left',
                                          left_on='UserID_bYear',right_index=True)
    
    ## add BD index column:
    final_BD_FD_converter['BD_index']=final_BD_FD_converter['BD'].str.replace('BD','')
    final_BD_FD_converter=final_BD_FD_converter.sort_values(by='BD')
    
    ## save final file:
    print 'final number of samples in the list is %s' %len(final_BD_FD_converter)

    print 'saving final files...'
    file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOn%s_RCfolder%s_%s.xlsx' %(mergeOn,
                                                                                                                                       readCountFolder,cdate)
    final_BD_FD_converter.to_excel(file1)

    file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOn%s_RCfolder%s_%s' %(mergeOn,
                                                                                                                                  readCountFolder,cdate)
    final_BD_FD_converter.to_pickle(file2)

   
    print 'Done!!!'
    
    return final_BD_FD_converter

    
    

## run fucnction:

In [ ]:
final_BD_FD_converter_YEAR=gen_BD_FD_conversion_file(mergeOn='Year',readCountFolder='Metabolon2')

In [ ]:
final_BD_FD_converter_YEAR=gen_BD_FD_conversion_file(mergeOn='Year',readCountFolder='AllSeqProjects')

In [3]:
final_BD_FD_converter_DATE=gen_BD_FD_conversion_file(mergeOn='Date',readCountFolder='AllSeqProjects')

number of rows without BD=35-those rows were removed
number of rows after addition of regNums is 1099


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/wisdom/python/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/usr/wisdom/python/lib/python2.7/threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/sbenari/workspace/PNP/tunneltoamazondb.py", line 33, in _start_tunnel
    ssh_tunnel.expect(pexpect.EOF)
  File "/usr/wisdom/python/lib/python2.7/site-packages/pexpect/spawnbase.py", line 321, in expect
    timeout, searchwindowsize, async)
  File "/usr/wisdom/python/lib/python2.7/site-packages/pexpect/spawnbase.py", line 345, in expect_list
    return exp.expect_loop(timeout)
  File "/usr/wisdom/python/lib/python2.7/site-packages/pexpect/expect.py", line 107, in expect_loop
    return self.timeout(e)
  File "/usr/wisdom/python/lib/python2.7/site-packages/pexpect/expect.py", line 70, in timeout
    raise TIMEOUT(msg)
TIMEOUT: Timeout exceeded.
command: /usr/bin/ssh
args: ['/usr/bin/ssh', 

1100
sample number is still 1099
extracted list of  2886 FD samples from DB
read count information for 6278 samples was loaded
maximal repeats per FD samples is 1 - SHOULD BE ONE! IF NOT, NEED TO CHECK IT
adding library preparation method:
saving BD and FD lists to excel files...
merging BD and FD samples...
after merging based on UserID and year, there are 1169 matches and 282 BD samples without     a match
merging unmatched samples based on UserID only...
coverter length is 1451
number of rows with noPostHGF info is 1143
number of unique BDs with postHGF info is 820
converter length is 1880
number of rows with noPostHGF info after new merging is 127
number of unique BDs with postHGF info after new merging is 1080
removing rows with no FD sample:
n samples after removal=1848
dropping duplicates
n samples after removal=1825


SampleFileFunctions.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  final_BD_FD_converter['SameYear'] = np.where(final_BD_FD_converter['Blood_Year']==final_BD_FD_converter['FD_Year'],1,0)
SampleFileFunctions.py:277: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  final_BD_FD_converter['SameDate'] = np.where(final_BD_FD_converter['Blood_Date']==final_BD_FD_converter['FD_Date'],1,0)


 n FDs that repeat severl times within the repeating BD: 19
n repeating BDs: 433
I checked multyBDperFD pairs in file final_BD_FD_converter_mergedOnYear_07032018 (see section 4 below) and saw that trying to resolve those pairs by using dates instead of years was not proved helpful. so i will leave these pairs as is.
final number of samples in the list is 1825
saving final files...
Done!!!


# develop

## extract blood DNA information:

In [ ]:
BDsamples = read_sql('select  DnaID,UserID,storageDT from Lab.hostdna', getengine())
BDsamples=BDsamples.rename(columns={'UserID':'oldUserID'})
BDsamples['DnaID_num']=BDsamples['DnaID'].str.replace('BD','').astype(np.float)
emptyBDs=BDsamples[BDsamples['DnaID'].isnull()]

print 'extracted list of  %s BD samples from DB' %len(BDsamples)
print 'last BD index is %s' %BDsamples['DnaID_num'].max()

BDsamples=BDsamples[BDsamples['DnaID'].notnull()] #remove rows with no BD
print 'number of rows without BD=%s-those rows were removed' %len(emptyBDs)

countingBDs=pd.DataFrame(BDsamples['DnaID_num'].value_counts()) #counting repeating BDs
repeatingBDs=countingBDs[countingBDs['DnaID_num']>1]
BDsamples=BDsamples.drop_duplicates(subset='DnaID') #removing BD duplicates
print 'BD samples repeating more than once=%s, duplicates were removed' %list(repeatingBDs.index)


print 'final number of rows should be the initial list minus empty BD rows and minus duplicate BD rows\nand equall to the last BD index'
print 'final number of rows is %s' %len(BDsamples)

BDsamples.head()

## correct UserIDs using correct RegNums:

In [ ]:
## correct UserIDs using correct RegNums:
#open the file with corrected userIDs for each BD number based on Daphna's correction for registration number:
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples'
CleanCorrectBloodDNASamples=pd.read_pickle(file1)
BD_correctReg=CleanCorrectBloodDNASamples[['DnaID','original registration code','correct registration code','correction status (reg number)']]




In [ ]:
# merge BD list with correct RegNum list:
BDwithCorrectReg=pd.merge(BDsamples,BD_correctReg,how='left',left_on='DnaID',right_on='DnaID')
BDwithCorrectReg['correct registration code']=BDwithCorrectReg['correct registration code'].fillna('unknown')
BDwithCorrectReg['correction status (reg number)']=BDwithCorrectReg['correction status (reg number)'].fillna('unknown')
print 'original BD sample list length is %s' %len(BDsamples)
print 'sample list length after merging is %s (SHOULD BE THE SAME AS ABOVE!)' %len(BDwithCorrectReg)



In [ ]:
BDwithCorrectReg[BDwithCorrectReg['correction status (reg number)']=='unknown'].head()

In [ ]:
# extract UserID-regNum from the database
UserIDs_RegNum=read_sql('select UserID,RegistrationCode from pnp.users', getengine())

#merge new UserID to BD sample according to the corrected regNum
BDwithCorrectUserID=pd.merge(BDwithCorrectReg,UserIDs_RegNum,how='left', 
                             left_on='correct registration code',
                            right_on='RegistrationCode')
#give old UserIDs and regNum to samples that don't have correct regNum:
noCorrectReg=len(BDwithCorrectUserID[BDwithCorrectUserID['correct registration code']=='unknown'])
print '%s samples dont have correct registration code and are given old UserID' %noCorrectReg
BDwithCorrectUserID['UserID']=np.where(BDwithCorrectUserID['correct registration code']=='unknown',
                                  BDwithCorrectUserID['oldUserID'],BDwithCorrectUserID['UserID'])


print 'sample number is still %s' %len(BDwithCorrectUserID)

BDwithCorrectUserID=BDwithCorrectUserID[['DnaID','storageDT', 'UserID','correct registration code','correction status (reg number)']]


In [ ]:
UserIDs_RegNum.head()

In [ ]:
#add regNum according to userID and use it only for samples with unknown regNum:
BDwithCorrectUserIDandRegs=pd.merge(BDwithCorrectUserID,UserIDs_RegNum,how='left',
                                   left_on='UserID',right_on='UserID')
BDwithCorrectUserIDandRegs.head()
BDwithCorrectUserIDandRegs['correct registration code']=np.where(BDwithCorrectUserIDandRegs['correct registration code']=='unknown',
                                  BDwithCorrectUserIDandRegs['RegistrationCode'],BDwithCorrectUserIDandRegs['correct registration code'])


In [ ]:
BDwithCorrectUserIDandRegs[BDwithCorrectUserIDandRegs['correction status (reg number)']=='unknown'].head()

In [ ]:
BDwithCorrectUserIDandRegs=BDwithCorrectUserIDandRegs.drop('RegistrationCode',axis=1)

## generate BD_year and BD_Date

In [ ]:
BDwithCorrectUserIDandRegs['storageDT']=BDwithCorrectUserIDandRegs['storageDT'].astype(str)
BDwithCorrectUserIDandRegs['Blood_Year']=BDwithCorrectUserIDandRegs['storageDT'].str.split('-').str[0]
BDwithCorrectUserIDandRegs['Blood_Date']=BDwithCorrectUserIDandRegs['storageDT'].str.split(' ').str[0]
BDwithCorrectUserIDandRegs['UserID']=BDwithCorrectUserIDandRegs['UserID'].astype(str)
BDwithCorrectUserIDandRegs['UserID']=BDwithCorrectUserIDandRegs['UserID'].str.split('.').str[0]
BDwithCorrectUserIDandRegs['UserID_bYear']=BDwithCorrectUserIDandRegs['UserID'].astype(str).str.cat(BDwithCorrectUserIDandRegs['Blood_Year'],sep='_')
BDwithCorrectUserIDandRegs['UserID_bDate']=BDwithCorrectUserIDandRegs['UserID'].astype(str).str.cat(BDwithCorrectUserIDandRegs['Blood_Date'],sep='_')
BDwithCorrectUserIDandRegs.head()

## extract FD table

In [ ]:
FDsamples = read_sql('select  DnaID,UserID,connectionID,storageDT,SPID,isGenotek,TubeID from Lab.dna', getengine())
print 'extracted list of  %s FD samples from DB' %len(FDsamples)
FDsamples.head()

In [ ]:
FDsamples['isGenotek'].value_counts(dropna=False)

In [ ]:
print len(FDsamples[FDsamples['SPID'].isnull()])
print len(FDsamples)

## generate FD year and date:

In [ ]:
FDsamples['storageDT']=FDsamples['storageDT'].astype(str)
FDsamples['FD_Year']=FDsamples['storageDT'].str.split('-').str[0]
FDsamples['FD_Date']=FDsamples['storageDT'].str.split(' ').str[0]
FDsamples['UserID']=FDsamples['UserID'].astype(str)
FDsamples['UserID']=FDsamples['UserID'].str.split('.').str[0]
FDsamples['UserID_fYear']=FDsamples['UserID'].astype(str).str.cat(FDsamples['FD_Year'],sep='_')
FDsamples['UserID_fDate']=FDsamples['UserID'].astype(str).str.cat(FDsamples['FD_Date'],sep='_')
FDsamples.head()

In [ ]:
FDsamples[FDsamples['UserID']=='2229']

### add read number information to FD samples:

In [ ]:

folder='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut'
filenames = [f for f in listdir(folder) if isfile(join(folder, f))]
filenames

In [ ]:
#load read number information for FD samples #####based on ReadCountSpidDF file!!####
f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountSpidDF.dat'
ReadCountDF=pd.read_pickle(f1)
print len(ReadCountDF)
ReadCountDF.head()

In [ ]:
ReadCountDF.columns.values

In [ ]:
#load read number information for FD samples:
# f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountDF.dat'
# ReadCountDF=pd.read_pickle(f1)
# print len(ReadCountDF)
# ReadCountDF.head()

In [ ]:
ReadCountDF=ReadCountDF.reset_index()
ReadCountDF.head()

In [ ]:
#check how many samples have read counts data:
print len(ReadCountDF)
print ReadCountDF.reset_index()['FD'].nunique()
print len(ReadCountDF[ReadCountDF['PostHGF'].notnull()])
print ReadCountDF[ReadCountDF['PostHGF'].notnull()].reset_index()['FD'].nunique()

In [ ]:
#check if there are FD samples without PostHGF info:
ReadCountDF[ReadCountDF['PostHGF'].isnull()]

In [ ]:
#check if there are FD samples repeating twice:
maxRepeatsPerFD=ReadCountDF.reset_index()['FD'].value_counts().sort_values().max()
print 'read count information for %s samples was loaded' %len(ReadCountDF)
print 'maximal repeats per FD samples is %s - SHOULD BE ONE! IF NOT, NEED TO CHECK IT' %maxRepeatsPerFD

no FD repeats more than once

In [ ]:
#extract only important information to merge with FD table:
ReadCountDFsmall=ReadCountDF.reset_index()[['FD','PostHGF','PostSubSamp']]
print len(ReadCountDFsmall[ReadCountDFsmall['PostHGF'].isnull()])
ReadCountDFsmall.head()

In [ ]:
'FD238' in list(ReadCountDFsmall['FD'])

In [ ]:
#merge read counts info with FD table:
FDsamples_withReads=pd.merge(FDsamples,ReadCountDFsmall,how='left',left_on='DnaID',right_on='FD')
FDsamples_withReads=FDsamples_withReads.drop('FD',axis=1)
print len(FDsamples)
print len(FDsamples_withReads)
FDsamples_withReads.head()


In [ ]:
FDsamples_withReads[FDsamples_withReads['DnaID']=='FD238']

#### try to merge unmerged samples based on SPID:
I could add read count info to 168 additional FD samples, but it seems problematic - it gives read counts of DIFFERENT FD sample from the same SPID.
not implemented at the moment

In [ ]:
# addInfo=0
# for n in FDsamples_withReads[FDsamples_withReads['PostHGF'].isnull()].index:
# #     print n
#     SPID=FDsamples_withReads.loc[n,'SPID']
#     FDinFDtable=FDsamples_withReads.loc[n,'DnaID']
    
#     if SPID in list(ReadCountDF['SPID']):
#         postHGF=ReadCountDF[ReadCountDF['SPID']==SPID].loc[:,'PostHGF']
#         PostSubSamp=ReadCountDF[ReadCountDF['SPID']==SPID].loc[:,'PostSubSamp']
#         FDinReadCountDF=ReadCountDF[ReadCountDF['SPID']==SPID].loc[:,'FD']
#         FDsamples_withReads.loc[n,'PostHGF']=postHGF.iloc[0]
#         FDsamples_withReads.loc[n,'PostSubSamp']=PostSubSamp.iloc[0]
# #         print n,postHGF.iloc[0],PostSubSamp.iloc[0]
#         addInfo=addInfo+1
#         print 'info added to SPID %s, FDinFDtable is %s and  FDinReadCountDF is %s' %(SPID,FDinFDtable,FDinReadCountDF)


In [ ]:
addInfo

In [ ]:
#check how many samples have read counts data:
print len(FDsamples_withReads)
print FDsamples_withReads['DnaID'].nunique()
print len(FDsamples_withReads[FDsamples_withReads['PostHGF'].notnull()])
print FDsamples_withReads[FDsamples_withReads['PostHGF'].notnull()]['DnaID'].nunique()

### add meeting location and sequencing machine information to FD samples:

NOT READY! DO NOT USE YET!!!!!!

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountDF.dat'
StoolMetadataDF=pd.read_pickle(f1)
print len(StoolMetadataDF)
StoolMetadataDF.head()

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/AllSeqProjects/DFOut/ReadCountSpidDF.dat'
ReadCountDF=pd.read_pickle(f1)
print len(ReadCountDF)
ReadCountDF.head()

In [ ]:
StoolMetadataDF.reset_index()['FD'].nunique()

In [ ]:
FDinConverter=list(final_BD_FD_converter_YEAR['FD'].unique())
FDinNewFile=list(StoolMetadataDF.reset_index()['FD'].unique())
FDinReadCount=list(ReadCountDF.reset_index()['FD'].unique())
FDinFDsamples_withReads=list(FDsamples_withReads.reset_index()['DnaID'].unique())
print len(FDinConverter)
print len(FDinNewFile)
print len(FDinReadCount)
print len(FDinFDsamples_withReads)

In [ ]:
'FD238' in FDinFDsamples_withReads

In [ ]:
'FD238' in FDinReadCount

In [ ]:
'FD238' in FDinNewFile

In [ ]:
ReadCountDF=ReadCountDF.reset_index()
ReadCountDF[ReadCountDF['FD']=='FD238']

In [ ]:
ReadCount_NewFile=list(set(FDinReadCount+FDinNewFile))
print len(ReadCount_NewFile)

In [ ]:
a=FDinConverter
b=FDinNewFile
print len(set(a).intersection(b))

In [ ]:
f1='/net/mraid08/export/jafar/Microbiome/Analyses/Metabolon2/DFOut/MeasurementsDF.dat'
MeasurementsDF=pd.read_pickle(f1)
print len(MeasurementsDF)
MeasurementsDF.head()

In [ ]:
FDsamples_withReads['TubeIDindex']=FDsamples_withReads['TubeID'].str.replace('fid','')
FDsamples_withReads.head()

In [ ]:
# extract meeting location informaiton for stool sample from Lab.stool:
StoolSamples = read_sql('select UserID, ConnectionID, ProductionDT,storageDT,FID1,FID2,BoxLocation,SPID from Lab.stool', getengine())
StoolSamples['TubeIDindex1']=StoolSamples['FID1'].str.replace('fid','',case=False)
StoolSamples['TubeIDindex2']=StoolSamples['FID2'].str.replace('fid','',case=False)
print len(StoolSamples)
StoolSamples.head()

In [ ]:
#extract year and date:
StoolSamples['storageDT']=StoolSamples['storageDT'].astype(str)
StoolSamples['stool_Year']=StoolSamples['storageDT'].str.split('-').str[0]
StoolSamples['stool_Date']=StoolSamples['storageDT'].str.split(' ').str[0]
StoolSamples['UserID']=StoolSamples['UserID'].astype(str)
StoolSamples['UserID']=StoolSamples['UserID'].str.split('.').str[0]
StoolSamples['UserID_sYear']=StoolSamples['UserID'].astype(str).str.cat(StoolSamples['stool_Year'],sep='_')
StoolSamples['UserID_sDate']=StoolSamples['UserID'].astype(str).str.cat(StoolSamples['stool_Date'],sep='_')
StoolSamples.head()

In [ ]:
#merge stool and FD samples on user+year:
FDsamples_withReads_withLocation=pd.merge(FDsamples_withReads,StoolSamples,how='left',
                                         left_on='UserID_fDate',right_on='UserID_sDate')
FDsamples_withReads_withLocation.head()

NOT GOOD TO MERGE ON DATE/YEAR+USER - TOO MANY UNCERTAINTIES!

## save BD and FD files

In [ ]:
print 'saving BD and FD lists to excel files...'
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/BDfile_%s.xlsx' %cdate
BDwithCorrectUserID.to_excel(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/FDfile_%s.xlsx' %cdate
FDsamples_withReads.to_excel(file2)

## MERGE FD and BD tables

In [ ]:
print 'merging BD and FD samples...'
BD_FD_converter=pd.merge(BDwithCorrectUserIDandRegs,FDsamples_withReads,how='left',left_on=['UserID','UserID_bYear'],
                        right_on=['UserID','UserID_fYear'])
BD_FD_converter.head()

In [ ]:
BD_FD_converter.tail()

In [ ]:
BD_FD_converter=BD_FD_converter.rename(columns={'DnaID_x':'BD','DnaID_y':'FD'})

In [ ]:
BD_FD_converter.head()

In [ ]:
# print 'after merging based on UserID and year, there are %s matches and %s BD samples without \
# a match' %(len(BD_FD_converter[BD_FD_converter['FD'].notnull()]),
#           len(BD_FD_converter[BD_FD_converter['FD'].isnull()]))


In [ ]:
print 'after merging based on UserID and Date, there are %s matches and %s BD samples without \
a match' %(len(BD_FD_converter[BD_FD_converter['FD'].notnull()]),
          len(BD_FD_converter[BD_FD_converter['FD'].isnull()]))


In [ ]:
BD_FD_converter[BD_FD_converter['FD'].isnull()].head()

In [ ]:
BD_FD_converter[BD_FD_converter['BD']=='BD345']

### merging unmatched BDs to FDs based on UserID only:
new strategy started on 16.3.18

In [ ]:
BDwithNoFDs=BD_FD_converter[BD_FD_converter['FD'].isnull()]
BDwithFDs=BD_FD_converter[BD_FD_converter['FD'].notnull()]

In [ ]:
BDwithNoFDs.head()

In [ ]:
BDwithNoFDs.index.nunique()

In [ ]:
print len(BDwithNoFDs)

In [ ]:
BDwithNoFDs['BD'].value_counts().max()

In [ ]:
converterLength=len(BD_FD_converter)
noPostHGF=len(BD_FD_converter[BD_FD_converter['PostHGF'].notnull()])
uniqueBDwithPostHGF=BD_FD_converter[BD_FD_converter['PostHGF'].notnull()]['BD'].nunique()
print 'coverter length is %s' %converterLength
print 'number of rows with noPostHGF info is %s' %noPostHGF
print 'number of unique BDs with postHGF info is %s' %uniqueBDwithPostHGF

In [ ]:
BD_FD_converter['Comment']=''
for BD in list(BDwithNoFDs['BD']):
    BDdf=BDwithNoFDs[BDwithNoFDs['BD']==BD]
    UserID=list(BDdf.loc[:,'UserID'])[0]
    n=BDdf.index
    BDtoMerge=BDwithCorrectUserIDandRegs[BDwithCorrectUserIDandRegs['DnaID']==BD]
#     print n.astype(int),BD, UserID
#     print 'length of BDtoMerge is %s and should be 1' %len(BDtoMerge)
    correspondingFDdf=FDsamples_withReads[FDsamples_withReads['UserID']==UserID]
    if len(correspondingFDdf)>0:
        newMerge=pd.merge(BDtoMerge,correspondingFDdf,how='left',
                          left_on='UserID',right_on='UserID')
        newMerge['Comment']='Not the same year!!'
        newMerge=newMerge.rename(columns={'DnaID_x':'BD','DnaID_y':'FD'})
#         print newMerge
#         print 'number of corresponding FD is %s' %len(correspondingFDdf)
#         print 'len of newMerge is %s' %len(newMerge)
#         print 'len of converter before changes is %s' %len(BD_FD_converter)
#         BD_FD_converter=BD_FD_converter.drop(n,axis=0)
        BD_FD_converter=pd.concat([BD_FD_converter,newMerge]) 
#         print 'len of converter after changes is %s' %len(BD_FD_converter)
#         print BD_FD_converter[BD_FD_converter['BD']==BD]
    


In [ ]:
converterLength=len(BD_FD_converter)
noPostHGF=len(BD_FD_converter[BD_FD_converter['PostHGF'].isnull()])
uniqueBDwithPostHGF=BD_FD_converter[BD_FD_converter['PostHGF'].notnull()]['BD'].nunique()
print 'converter length is %s' %converterLength
print 'number of rows with noPostHGF info after new merging is %s' %noPostHGF
print 'number of unique BDs with postHGF info after new merging is %s' %uniqueBDwithPostHGF

In [ ]:
BD_FD_converter[BD_FD_converter['BD']=='BD345']

In [ ]:
# # go over BD samples without match FD and match FD based on userID only
# BDwithNoFDs=BD_FD_converter[BD_FD_converter['FD'].isnull()]
# BDwithFDs=BD_FD_converter[BD_FD_converter['FD'].notnull()]


# BD_FD_converter['Comment']=''
# for n in BD_FD_converter[BD_FD_converter['FD'].isnull()].index:
#         print 'n=%s' %n
#         userID=BD_FD_converter.loc[n,'UserID']
#         relevantDF=FDsamples[FDsamples['UserID']==userID] #generate a df with all FD samples for the specific userID
#         relevantDF=relevantDF.set_index('DnaID')
#         print len(relevantDF)
#         DFlength=len(relevantDF)
#         FDlist=list(relevantDF.index)
#         relevantFDlist=[x for x in FDlist if x not in list(BDwithFDs['FD'])] #use only FDs that belong to the UserID and ARE NOT
#                                                                      # connected already with another BD
#         relevantDF2=relevantDF.loc[relevantFDlist,:]
#         relevantDF2=relevantDF2.reset_index()
#         print len(relevantDF2)
#         FDlist2=list(relevantDF2['DnaID'])
#         UserFlist=list(relevantDF2['UserID'])
#         FDyearList=list(relevantDF2['FD_Year'])
#         UserID_fYearList=list(relevantDF2['UserID_fYear'])
#         FDDateList=list(relevantDF2['FD_Date'])
#         UserID_fDateList=list(relevantDF2['UserID_fDate'])
#         BDlist=BD_FD_converter.loc[n,'BD']
#         Blood_Yearlist=BD_FD_converter.loc[n,'Blood_Year']
#         UserID_bYearlist=BD_FD_converter.loc[n,'UserID_bYear']
#         Blood_Datelist=BD_FD_converter.loc[n,'Blood_Date']
#         UserID_bDatelist=BD_FD_converter.loc[n,'UserID_bDate']
#         correctRegList=BD_FD_converter.loc[n,'correct registration code']
#         correctStatusList=BD_FD_converter.loc[n,'correction status (reg number)']
#         newDF=pd.DataFrame({'BD':BDlist,
#                             'UserID':userID,
#                             'FD':FDlist2,
#                             'FD_Year':FDyearList,
#                             'UserID_fYear':UserID_fYearList,
#                             'FD_Date':FDDateList,
#                             'UserID_fDate':UserID_fDateList,
#                             'Blood_Year':Blood_Yearlist,
#                             'UserID_bYear':UserID_bYearlist,
#                             'Blood_Date':Blood_Datelist,
#                             'UserID_bDate':UserID_bDatelist,
#                             'correct registration code':correctRegList,
#                             'correction status (reg number)':correctStatusList,
#                             'Comment':'Not the same year!!'})
        
# #         print newDF
# #         print BD_FD_converter.columns.values
#         BD_FD_converter=pd.concat([BD_FD_converter,newDF])

In [ ]:
BD_FD_converter.head()

In [ ]:
# # remove repeating FD samples:
# BD_FD_converter=BD_FD_converter.reset_index()
# print 'removing repeating FD samples:'
# print 'n samples before removal: %s' %len(BD_FD_converter)
# groupbyFD=BD_FD_converter.groupby('FD')['BD'].nunique()
# moreThan1=groupbyFD[groupbyFD>1]
# moreThan1

# BD_FD_converter2=BD_FD_converter
# for FD in moreThan1.index:
#     df=BD_FD_converter[BD_FD_converter['FD']==FD]
# #     print FD, len(df),len(df[df['Comment']==''])
#     if len(df)>1 and len(df[df['Comment']==''])>0:
# #         print df[df['Comment']=='Not the same year!!']
#         for n in df[df['Comment']=='Not the same year!!'].index:
#             print 'dropping the following row:'
#             print BD_FD_converter2.loc[n,:]
#             BD_FD_converter2=BD_FD_converter2.drop(n,axis=0)
            
        
# print 'n samples after removal: %s' %len(BD_FD_converter2)

In [ ]:
final_BD_FD_converter2=BD_FD_converter[BD_FD_converter['FD'].notnull()]
final_BD_FD_converter2.head()

In [ ]:
print len(BD_FD_converter)
print len(final_BD_FD_converter2)

In [ ]:
print BD_FD_converter['BD'].nunique()
print final_BD_FD_converter2['BD'].nunique()

In [ ]:
final_BD_FD_converter=final_BD_FD_converter2[['BD', 'Blood_Year', 'Blood_Date','Comment', 'FD', 'FD_Year','FD_Date', u'SPID', u'UserID',
                                            'correct registration code','correction status (reg number)','UserID_bYear', 'UserID_fYear',
                                             'UserID_bDate', 'UserID_fDate',u'connectionID', u'isGenotek',
                                             'PostHGF','PostSubSamp']]

final_BD_FD_converter.head()

### work on FD-BD pairs from the same year:


I checked those pairs in file final_BD_FD_converter_mergedOnYear_07032018 (see section 4 below) and saw that tring to resolve those pairs by using dates instead of years is not proved helpful. so i'll leave this pairs as is. 

In [ ]:
pd.DataFrame(final_BD_FD_converter.groupby('FD')['BD'].nunique()).head()

In [ ]:
countBDperFD=pd.DataFrame(final_BD_FD_converter.groupby('FD')['BD'].nunique())

In [ ]:
multyBDperFD=countBDperFD[countBDperFD['BD']>1]
print len(multyBDperFD)

In [ ]:
for FD in multyBDperFD.index:
    df=final_BD_FD_converter[final_BD_FD_converter['FD']==FD]
    print FD
    print df
#     for n in df.index:
#         BD=df.loc[n,'BD']
#         Blood_Date=df.loc[n,'Blood_Date']
#         FD_Date=df.loc[n,'FD']
        

In [ ]:
final_BD_FD_converter['correction status (reg number)'].unique()

In [ ]:
print len(final_BD_FD_converter)

## add StudyType

In [ ]:
#get studyTypeID and UserID information:
studyTypeSPIDs = read_sql('select  * from pnp.spid', getengine())
studyTypeSPIDs.head()

In [ ]:
studyTypeSPIDs['SPID_Year']=studyTypeSPIDs['Start_date'].astype(str).str.split('-').str[0]
studyTypeSPIDs['SPID_Date']=studyTypeSPIDs['Start_date'].astype(str).str.split(' ').str[0]
studyTypeSPIDs['UserID']=studyTypeSPIDs['UserID'].astype(str)
studyTypeSPIDs['UserID']=studyTypeSPIDs['UserID'].str.split('.').str[0]
studyTypeSPIDs['UserID_SPIDYear']=studyTypeSPIDs['UserID'].astype(str).str.cat(studyTypeSPIDs['SPID_Year'],sep='_')
studyTypeSPIDs['UserID_SPIDDate']=studyTypeSPIDs['UserID'].astype(str).str.cat(studyTypeSPIDs['SPID_Date'],sep='_')
studyTypeSPIDs.head()

In [ ]:
studyTypeSPIDs_small=studyTypeSPIDs[['StudyTypeID','UserID_SPIDYear']]

In [ ]:
studyTypelists=studyTypeSPIDs_small.groupby('UserID_SPIDYear').agg(lambda x: x.unique().tolist())

In [ ]:
studyTypelists[200:205]

In [ ]:
final_BD_FD_converter=pd.merge(final_BD_FD_converter,studyTypelists,how='left',
                                          left_on='UserID_bYear',right_index=True)
final_BD_FD_converter.head()

## add postHGF info to FD samples based on registrant numbers:
it seems that registrant numbers in the read count table are not similar to regNum in the converter file, so I dropped this direction

## save final file:


In [ ]:
print 'final number of samples in the list is %s' %len(final_BD_FD_converter)

print 'saving final files...'
file1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_%s.xlsx' %cdate 
final_BD_FD_converter.to_excel(file1)

file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_%s' %cdate
final_BD_FD_converter.to_pickle(file2)



In [ ]:
final_BD_FD_converter.groupby('FD')['BD'].nunique().sort_values(ascending=False)[:10]

In [ ]:
BDperFD=pd.DataFrame(final_BD_FD_converter.groupby('FD')['BD'].nunique().sort_values(ascending=False))
BDperFD.head()

In [ ]:
print len(BDperFD[BDperFD['BD']>1])

# compare converter file quality:

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter' 
final_BD_FD_converter=pd.read_pickle(file2)
print len(final_BD_FD_converter)
final_BD_FD_converter.head()

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_07032018' 
final_BD_FD_converter_mergedOnYear_07032018=pd.read_pickle(file2)
print len(final_BD_FD_converter_mergedOnYear_07032018)
final_BD_FD_converter_mergedOnYear_07032018.head()

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnDate_07032018' 
final_BD_FD_converter_mergedOnDate_07032018=pd.read_pickle(file2)
print len(final_BD_FD_converter_mergedOnDate_07032018)
final_BD_FD_converter_mergedOnDate_07032018.head()

In [ ]:
file2='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_14032018' 
final_BD_FD_converter_mergedOnYear_14032018=pd.read_pickle(file2)
print len(final_BD_FD_converter_mergedOnYear_14032018)
final_BD_FD_converter_mergedOnYear_14032018.head()

In [ ]:
print final_BD_FD_converter['BD'].nunique()

In [ ]:
print final_BD_FD_converter_mergedOnYear_07032018['BD'].nunique()

In [ ]:
print final_BD_FD_converter_mergedOnYear_14032018['BD'].nunique()

In [ ]:
final_BD_FD_converter[final_BD_FD_converter['UserID'].isnull()]

In [ ]:
final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['UserID'].isnull()]

In [ ]:
final_BD_FD_converter_mergedOnYear_14032018[final_BD_FD_converter_mergedOnYear_14032018['UserID'].isnull()]

In [ ]:
print len(final_BD_FD_converter[final_BD_FD_converter['SPID'].isnull()])

In [ ]:
print len(final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['SPID'].isnull()])

In [ ]:
print len(final_BD_FD_converter_mergedOnYear_14032018[final_BD_FD_converter_mergedOnYear_14032018['SPID'].isnull()])

In [ ]:
final_BD_FD_converter[final_BD_FD_converter['FD'].isnull()]

In [ ]:
print len(final_BD_FD_converter[final_BD_FD_converter['connectionID'].isnull()])

In [ ]:
print len(final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['connectionID'].isnull()])

In [ ]:
print len(final_BD_FD_converter_mergedOnYear_14032018[final_BD_FD_converter_mergedOnYear_14032018['connectionID'].isnull()])

In [ ]:
countBDperFD_07032018=pd.DataFrame(final_BD_FD_converter_mergedOnYear_07032018.groupby('FD')['BD'].nunique())

In [ ]:
multyBDperFD_07032018=countBDperFD_07032018[countBDperFD_07032018['BD']>1]
print len(multyBDperFD_07032018)

In [ ]:
countBDperFD_14032018=pd.DataFrame(final_BD_FD_converter_mergedOnYear_14032018.groupby('FD')['BD'].nunique())

In [ ]:
multyBDperFD_14032018=countBDperFD_14032018[countBDperFD_14032018['BD']>1]
print len(multyBDperFD_14032018)

In [ ]:
for FD in multyBDperFD_07032018.index:
    df=final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['FD']==FD]
    print FD
    print df
#     for n in df.index:
#         BD=df.loc[n,'BD']
#         Blood_Date=df.loc[n,'Blood_Date']
#         FD_Date=df.loc[n,'FD']
        

In [ ]:
for FD in multyBDperFD_14032018.index:
    df=final_BD_FD_converter_mergedOnYear_14032018[final_BD_FD_converter_mergedOnYear_14032018['FD']==FD]
    print FD
    print df
#     for n in df.index:
#         BD=df.loc[n,'BD']
#         Blood_Date=df.loc[n,'Blood_Date']
#         FD_Date=df.loc[n,'FD']
        

In [ ]:
countBDperFD_07032018_date=pd.DataFrame(final_BD_FD_converter_mergedOnDate_07032018.groupby('FD')['BD'].nunique())

In [ ]:
countBDperFD_14032018_date=pd.DataFrame(final_BD_FD_converter_mergedOnDate_14032018.groupby('FD')['BD'].nunique())

In [ ]:
multyBDperFD_07032018_date=countBDperFD_07032018_date[countBDperFD_07032018_date['BD']>1]
print len(multyBDperFD_07032018_date)

In [ ]:
multyBDperFD_14032018_date=countBDperFD_14032018_date[countBDperFD_14032018_date['BD']>1]
print len(multyBDperFD_14032018_date)

In [ ]:
for FD in multyBDperFD_07032018_date.index:
    df=final_BD_FD_converter_mergedOnDate_07032018[final_BD_FD_converter_mergedOnDate_07032018['FD']==FD]
    print FD
    print df
#     for n in df.index:
#         BD=df.loc[n,'BD']
#         Blood_Date=df.loc[n,'Blood_Date']
#         FD_Date=df.loc[n,'FD']
        

### explore missing identifiers in the mergeOn=Date file:

In [ ]:
final_BD_FD_converter_mergedOnYear_07032018.head()

In [ ]:
noDate=final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['Blood_Date'].isnull()]
print len(noDate)
noDate.head()

In [ ]:
noFD=final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['FD'].isnull()]
print len(noFD)
noFD.head()

In [ ]:
noSPID=final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['SPID'].isnull()]
print len(noSPID)
noSPID.head()

In [ ]:
noConnection=final_BD_FD_converter_mergedOnYear_07032018[final_BD_FD_converter_mergedOnYear_07032018['connectionID'].isnull()]
print len(noConnection)
noConnection.head()

# add library preparation method to FD samples:

In [ ]:
f1='%s/Sample files/all_fds_barcode_df.dat' %MyPath
libPrep=pd.read_pickle(f1)
libPrep.Barcode.value_counts()
# libPrep.head()

In [ ]:
libPrep.shape

# some checks - 130518

In [ ]:
file1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/CleanCorrectBloodDNASamples'
CleanCorrectBloodDNASamples = pd.read_pickle(file1)

In [ ]:
CleanCorrectBloodDNASamples.head()

In [ ]:
CleanCorrectBloodDNASamples[CleanCorrectBloodDNASamples['DnaID']=='BD38']

In [ ]:
UserIDs_RegNum = read_sql('select UserID,RegistrationCode from pnp.users', getengine())

In [ ]:
UserIDs_RegNum.head()

In [ ]:
UserIDs_RegNum[UserIDs_RegNum['RegistrationCode']==541391]

In [ ]:
BDsamples = read_sql('select  DnaID,UserID,storageDT from Lab.hostdna', getengine())

In [ ]:
BDsamples.head()

In [ ]:
BDsamples[BDsamples['DnaID']=='BD38']

In [ ]:
BDsamples[BDsamples['UserID']==11343]

In [ ]:
import time
cdate = str(time.strftime("%d%m%Y"))
cdate

# # extract blood DNA information:
BDsamples = read_sql('select  DnaID,UserID,storageDT from Lab.hostdna', getengine())
BDsamples = BDsamples.rename(columns={'UserID':'oldUserID'})
BDsamples['DnaID_num'] = BDsamples['DnaID'].str.replace('BD', '').astype(np.float)
emptyBDs = BDsamples[BDsamples['DnaID'].isnull()]
BDsamples = BDsamples[BDsamples['DnaID'].notnull()] 

In [ ]:
print BDsamples[BDsamples['DnaID']=='BD937']
print BDsamples[BDsamples['DnaID']=='BD938']

In [ ]:
print 'number of rows without BD=%s-those rows were removed' % len(emptyBDs)

countingBDs = pd.DataFrame(BDsamples['DnaID_num'].value_counts())  # counting repeating BDs
repeatingBDs = countingBDs[countingBDs['DnaID_num'] > 1]
print repeatingBDs
BDsamples = BDsamples.drop_duplicates(subset='DnaID')  # removing BD duplicates

In [ ]:
print BDsamples[BDsamples['DnaID']=='BD937']
print BDsamples[BDsamples['DnaID']=='BD938']

In [ ]:
UserIDs_RegNum = read_sql('select UserID,RegistrationCode from pnp.users', getengine())

In [ ]:
# merge regNums to BD samples:
BDsamples_withOldIDs=pd.merge(BDsamples,UserIDs_RegNum,how='left',left_on='oldUserID',right_on='UserID')
BDsamples_withOldIDs=BDsamples_withOldIDs.drop('UserID', axis=1)
BDsamples_withOldIDs=BDsamples_withOldIDs.rename(columns={'RegistrationCode':'old_RegistrationCode'})
print len(BDsamples_withOldIDs)
BDsamples_withOldIDs.head()

In [ ]:
from ShaniBA.SampleLists.correctingRegNumsFromDaphna import getQCedBloodRegNumByRegNum


In [ ]:
BDsamples_corrected=BDsamples_withOldIDs

BDsamples_corrected['corrected_regCode']=BDsamples_corrected['old_RegistrationCode'].apply(lambda x:getQCedBloodRegNumByRegNum(x))
BDsamples_corrected.head(10)



In [ ]:
print len(BDsamples_corrected)

In [ ]:
print BDsamples_corrected[BDsamples_corrected['corrected_regCode']!=BDsamples_corrected['old_RegistrationCode']].head()
print len(BDsamples_corrected[BDsamples_corrected['corrected_regCode']!=BDsamples_corrected['old_RegistrationCode']])

In [ ]:
BDsamples_corrected['correction status (reg number)']=np.where(BDsamples_corrected['corrected_regCode']!=BDsamples_corrected['old_RegistrationCode'],'corrected','same')
BDsamples_corrected['correction status (reg number)']=np.where(BDsamples_corrected['corrected_regCode'].isnull(),'unknown',BDsamples_corrected['correction status (reg number)'])
BDsamples_corrected.head()

In [ ]:
BDsamples_corrected['correction status (reg number)'].value_counts(dropna=False)

In [ ]:
BDsamples_corrected['corrected_regCode']=np.where(BDsamples_corrected['corrected_regCode'].isnull(),BDsamples_corrected['old_RegistrationCode'],BDsamples_corrected['corrected_regCode'])

In [ ]:
print BDsamples_corrected[BDsamples_corrected['DnaID']=='BD937']
print BDsamples_corrected[BDsamples_corrected['DnaID']=='BD938']

In [ ]:
# add correct UserID:
BDsamples_corrected_withUserID=pd.merge(BDsamples_corrected,UserIDs_RegNum,how='left',left_on='corrected_regCode',right_on='RegistrationCode')
print len(BDsamples_corrected_withUserID)
BDsamples_corrected_withUserID.head()


In [ ]:
BDsamples_corrected_withUserID[BDsamples_corrected_withUserID['correction status (reg number)']!='same'].head(10)

In [ ]:
print len(BDsamples_corrected_withUserID[BDsamples_corrected_withUserID['UserID'].isnull()])
print len(BDsamples_corrected_withUserID)

In [ ]:
BDsamples_corrected_withUserID=BDsamples_corrected_withUserID[~((BDsamples_corrected_withUserID['UserID']==11343)&(BDsamples_corrected_withUserID['corrected_regCode']==541391))]
print 'sample number is still %s' % len(BDsamples_corrected_withUserID)

BDsamples_corrected_withUserID = BDsamples_corrected_withUserID[['DnaID', 'storageDT', 'UserID', 'corrected_regCode', 'correction status (reg number)']]

In [ ]:




# extract UserID-regNum from the database
UserIDs_RegNum = read_sql('select UserID,RegistrationCode from pnp.users', getengine())

# merge new UserID to BD sample according to the corrected regNum
BDwithCorrectUserID = pd.merge(BDwithCorrectReg, UserIDs_RegNum, how='left',
                             left_on='correct registration code',
                            right_on='RegistrationCode')
# give old UserIDs and regNum to samples that don't have correct regNum:
noCorrectReg = len(BDwithCorrectUserID[BDwithCorrectUserID['correct registration code'] == 'unknown'])
print '%s samples dont have correct registration code and are given old UserID' % noCorrectReg
BDwithCorrectUserID['UserID'] = np.where(BDwithCorrectUserID['correct registration code'] == 'unknown',
                                  BDwithCorrectUserID['oldUserID'], BDwithCorrectUserID['UserID'])

BDwithCorrectUserID=BDwithCorrectUserID[~((BDwithCorrectUserID['UserID']==11343)&(BDwithCorrectUserID['correct registration code']==541391))]
print 'sample number is still %s' % len(BDwithCorrectUserID)

BDwithCorrectUserID = BDwithCorrectUserID[['DnaID', 'storageDT', 'UserID', 'correct registration code', 'correction status (reg number)']]

# add regNum according to userID and use it only for samples with unknown regNum:
BDwithCorrectUserIDandRegs = pd.merge(BDwithCorrectUserID, UserIDs_RegNum, how='left',
                                   left_on='UserID', right_on='UserID')
BDwithCorrectUserIDandRegs['correct registration code'] = np.where(BDwithCorrectUserIDandRegs['correct registration code'] == 'unknown',
                                  BDwithCorrectUserIDandRegs['RegistrationCode'], BDwithCorrectUserIDandRegs['correct registration code'])

BDwithCorrectUserIDandRegs = BDwithCorrectUserIDandRegs.drop('RegistrationCode', axis=1)